In [ ]:
import pandas as pd
import numpy as np

from pathlib import Path
from datetime import datetime
from one.api import ONE

In [ ]:
one = ONE()

In [ ]:
ROOT_EXPERIMENTAL_FOLDER = Path.cwd().joinpath('data')

In [ ]:
session_dict = dict(subject='xxx', number=1, lab='YangLab', task_protocol='pacman_stage_001',
                    project="pacman", start_time=str(datetime.now()), users=['admin'])
session = one.alyx.rest('sessions', 'create', data=session_dict)
eid = session['id']

In [ ]:
session_path = ROOT_EXPERIMENTAL_FOLDER.joinpath(
    session['lab'], 'Subjects', session['subject'], session['start_time'][:10], str(session['number']).zfill(3))
alf_path = session_path.joinpath('alf') 
alf_path.mkdir(parents=True, exist_ok=True)

In [3]:
ntrials = 400
x_mock_001 = np.linspace(0, 4 * np.pi, ntrials)
y_mock_001 = np.sin(x_mock_001)

mock_001 = pd.DataFrame({
    'x': x_mock_001,
    'y': y_mock_001
})

mock_001.head()

,x,y
0,0.000000,0.000000
1,0.063148,0.063106
2,0.126295,0.125960
3,0.189443,0.188312
4,0.252590,0.249913


In [ ]:
file_path = alf_path.joinpath('mock.table.pqt')
mock_001.to_parquet(file_path)

In [ ]:
from data_transform import SmbClient
smbclient = SmbClient(r"address", "admin", "admin")
smbclient.upload(file_path, file_path)

In [ ]:
r = {'created_by': 'admin',
     'path': session_path.relative_to((session_path.parents[2])).as_posix(),
     'filenames': ['alf/trials.table.pqt'],
     'name': 'local'
     }
response = one.alyx.rest('register-file', 'create', data=r, no_cache=True)